# Prise des images

Importation des modules

In [ ]:
import os
import cv2

from enum import Enum
from functools import reduce

Chargement de la configuration

In [ ]:
import json

with open('config.json', 'r') as f:
   config = json.load(f)

DATA_DIR = config['dataset']['dir']
DATASET_SIZE = config['dataset']['size']
DATASET_DEFINITION = config['dataset']['definition']
DATASET_DEFINITION_KEYS = list(DATASET_DEFINITION.keys())

Fonction permettant de créer un dossier s'il n'existe pas

In [ ]:
def ensure_dir_exists(path):
    os.makedirs(path, exist_ok=True)

Fonction permettant de donner le fichier avec le nombre le plus élevé

In [ ]:
def calculate_offset(path):
   filenames = os.listdir(path)
   if not filenames:
      return 0
   
   file_numbers = [int(os.path.splitext(filename)[0]) for filename in filenames]
   return reduce(max, file_numbers) + 1

Fonction permettant d'afficher du texte centré

In [ ]:
def show_text_center(frame, text, font = cv2.FONT_HERSHEY_SIMPLEX, font_scale = 1.3, font_thickness = 3, font_color = (25, 135, 25), offset = 130, padding = 15):    
    text_size, _ = cv2.getTextSize(text, font, font_scale, font_thickness)
    text_position = ((frame.shape[1] - text_size[0]) // 2, frame.shape[0] - text_size[1] - offset)
    cv2.rectangle(frame, (text_position[0] - padding, text_position[1] - text_size[1] - padding), 
             (text_position[0] + text_size[0] + padding, text_position[1] + padding), (240, 240, 240), -1)
    cv2.putText(frame, text, text_position, font, font_scale, font_color, font_thickness, cv2.LINE_AA)

Attendre que l'utilisateur appuie sur une touche

In [ ]:
class Status(Enum):
    CONTINUE = 0
    EXIT = 1
    SKIP = 2

def wait_for_user_input(dataset_name):
    while True:
        _, frame = cap.read()
        show_text_center(frame, f"Appuyer sur la touche '<ESPACE>' pour enregistrer '{dataset_name}'")
        show_text_center(frame, f"Appuyer sur la touche 'S' passer au dataset suivant", offset=60 , font_color=(180, 85, 25))
        cv2.imshow('frame', frame)
        user_input = cv2.waitKey(25)
        if user_input == ord(' '):
            return Status.CONTINUE
        elif user_input == ord('q'):
            return Status.EXIT
        elif user_input == ord('s'):
            return Status.SKIP

Capturer et sauvegarder les différentes images

La copie de l'image avant d'ajouter, le texte permet de ne pas afficher le texte lors de la sauvegarde de l'image.

In [ ]:
def capture_images(folder, offset):
    for counter in range(DATASET_SIZE):
        _, frame = cap.read()
        clean_frame = frame.copy()
        show_text_center(frame, f"Enregistrement en cours ({counter + 1}/{DATASET_SIZE})", offset=60 , font_color=(40, 150, 215))
        cv2.imshow('frame', frame)
        cv2.waitKey(25)
        cv2.imwrite(os.path.join(DATA_DIR, str(folder), f"{counter + offset}.jpg"), clean_frame)

Pour chaque définition du dataset, on capture et on sauvegarde les images

In [ ]:
cap = cv2.VideoCapture(0)

ensure_dir_exists(DATA_DIR)
for dir_ in DATASET_DEFINITION_KEYS:
    ensure_dir_exists(os.path.join(DATA_DIR, str(dir_)))
    
    offset = calculate_offset(os.path.join(DATA_DIR, str(dir_)))
    
    user_input = wait_for_user_input(DATASET_DEFINITION[dir_])
    if user_input == Status.EXIT:
        break
    elif user_input == Status.SKIP:
        continue
    capture_images(dir_, offset)

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)